# Lecture 2 - Permutation Importance

## Giới Thiệu

Một trong những câu hỏi cơ bản nhất mà chúng ta có thể đặt ra về một mô hình là: Đặc trưng nào có tác động lớn nhất đến dự đoán?

Khái niệm này được gọi là **tầm quan trọng của đặc trưng** (*feature importance*).

Có nhiều cách để đo lường tầm quan trọng của đặc trưng. Một số phương pháp trả lời các phiên bản khác nhau của câu hỏi trên theo cách tinh tế. Một số phương pháp khác có những hạn chế đã được ghi nhận.

Trong bài học này, chúng ta sẽ tập trung vào **tầm quan trọng hoán vị** (*permutation importance*). So với hầu hết các phương pháp khác, phương pháp này có các ưu điểm:

- tính toán nhanh,
- được sử dụng rộng rãi và dễ hiểu,
- nhất quán với các thuộc tính mong muốn của một thước đo tầm quan trọng của đặc trưng.

## Cách Hoạt Động

Tầm quan trọng hoán vị sử dụng mô hình theo cách khác biệt so với những gì bạn đã thấy trước đây, và nhiều người ban đầu có thể cảm thấy khó hiểu. Vì vậy, chúng ta sẽ bắt đầu với một ví dụ để làm rõ khái niệm này.

Giả sử chúng ta có dữ liệu với định dạng sau:

![Data](https://storage.googleapis.com/kaggle-media/learn/images/wjMAysV.png)

Chúng ta muốn dự đoán chiều cao của một người khi họ 20 tuổi, dựa vào dữ liệu có sẵn khi họ 10 tuổi.

Dữ liệu của chúng ta bao gồm các đặc trưng hữu ích (*chiều cao lúc 10 tuổi*), các đặc trưng có ít khả năng dự đoán (*số lượng tất sở hữu*), cũng như một số đặc trưng khác mà chúng ta sẽ không tập trung vào trong phần giải thích này.

**Tầm quan trọng hoán vị được tính toán sau khi mô hình đã được huấn luyện.** Vì vậy, chúng ta không thay đổi mô hình hoặc thay đổi dự đoán mà nó đưa ra cho một giá trị chiều cao, số lượng tất, v.v.

Thay vào đó, chúng ta đặt câu hỏi: Nếu tôi xáo trộn ngẫu nhiên một cột duy nhất của dữ liệu kiểm tra, giữ nguyên mục tiêu và tất cả các cột khác, điều đó sẽ ảnh hưởng như thế nào đến độ chính xác của mô hình khi dự đoán trên dữ liệu đã bị xáo trộn?

![Shuffle](https://storage.googleapis.com/kaggle-media/learn/images/h17tMUU.png)

Việc xáo trộn ngẫu nhiên một cột duy nhất có thể dẫn đến dự đoán kém chính xác hơn, vì dữ liệu kết quả không còn tương ứng với bất cứ điều gì đã quan sát được trong thực tế. Độ chính xác của mô hình sẽ bị ảnh hưởng nghiêm trọng nếu chúng ta xáo trộn một cột mà mô hình phụ thuộc nhiều vào đó để dự đoán. Trong trường hợp này, việc xáo trộn `chiều cao lúc 10 tuổi` sẽ gây ra dự đoán rất tệ. Nếu chúng ta xáo trộn `số lượng tất sở hữu`, thì dự đoán sẽ không bị ảnh hưởng nhiều.

Với hiểu biết này, quá trình thực hiện như sau:

1. Có một mô hình đã được huấn luyện.
2. Xáo trộn các giá trị trong một cột duy nhất, sau đó tạo dự đoán với tập dữ liệu mới này. Sử dụng dự đoán đó cùng với giá trị mục tiêu thực tế để tính toán mức độ suy giảm hiệu suất do xáo trộn. Mức suy giảm này đo lường tầm quan trọng của biến bạn vừa xáo trộn.
3. Trả dữ liệu về thứ tự ban đầu (hoàn tác việc xáo trộn ở bước 2). Sau đó lặp lại bước 2 với cột tiếp theo trong tập dữ liệu, cho đến khi bạn tính toán xong tầm quan trọng của từng cột.

## Ví Dụ Mã Code

Ví dụ của chúng ta sẽ sử dụng một mô hình dự đoán liệu một đội bóng đá có giành được danh hiệu "Cầu thủ xuất sắc nhất trận" (*Man of the Game*) dựa trên các thống kê của đội hay không. Danh hiệu này được trao cho cầu thủ chơi hay nhất trận đấu. Việc xây dựng mô hình không phải là trọng tâm của chúng ta trong phần này, nên đoạn mã dưới đây sẽ tải dữ liệu và tạo một mô hình cơ bản.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# Đọc dữ liệu
data = pd.read_csv('FIFA 2018 Statistics.csv')

# Biến mục tiêu: Chuyển đổi từ 'Yes'/'No' sang nhị phân (0/1)
y = (data['Man of the Match'] == "Yes").astype(int)

# Chọn các cột có kiểu dữ liệu số nguyên (int) hoặc thực (float)
feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.float64]]
X = data[feature_names]

# Chia tập dữ liệu thành train/test
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Huấn luyện mô hình RandomForest
my_model = RandomForestClassifier(n_estimators=100, random_state=0)
my_model.fit(train_X, train_y)

# 🟢 Tính toán Permutation Importance
perm_importance = permutation_importance(my_model, val_X, val_y, n_repeats=10, random_state=1, scoring="accuracy")

# Chuyển kết quả thành DataFrame để dễ đọc
importances = pd.DataFrame({
    "Feature": val_X.columns,
    "Importance": perm_importance.importances_mean,
    "Std Dev": perm_importance.importances_std
}).sort_values(by="Importance", ascending=False)

# 🟢 Hiển thị kết quả
print(importances)  # Nếu chạy trên terminal
# display(importances)  # Nếu chạy trên Jupyter Notebook

# 🟢 Xuất ra file CSV (nếu cần)
importances.to_csv("permutation_importance.csv", index=False)
print("File permutation_importance.csv đã được lưu thành công!")

                   Feature  Importance   Std Dev
8               Free Kicks    0.109375  0.037630
17                1st Goal    0.084375  0.060998
6                  Corners    0.068750  0.023385
5                  Blocked    0.062500  0.036975
0              Goal Scored    0.059375  0.054932
10         Pass Accuracy %    0.034375  0.025958
11                  Passes    0.031250  0.024206
20           Own goal Time    0.031250  0.019764
3                On-Target    0.025000  0.027243
1        Ball Possession %    0.025000  0.030619
4               Off-Target    0.021875  0.024407
9                    Saves    0.015625  0.028811
2                 Attempts    0.015625  0.037630
7                 Offsides    0.006250  0.033657
16                     Red    0.000000  0.000000
19               Own goals    0.000000  0.000000
13         Fouls Committed   -0.003125  0.021875
15            Yellow & Red   -0.003125  0.009375
18            Goals in PSO   -0.003125  0.009375
12  Distance Covered

Dưới đây là cách tính toán và hiển thị tầm quan trọng của đặc trưng bằng thư viện [eli5](https://eli5.readthedocs.io/en/latest/):
- Tính tới hiện tại thì eli5 chưa có bản cập nhật phù hợp với phiên bản mới nhất của scikit-learn

## Giải Thích Tầm Quan Trọng Hoán Vị (*Permutation Importances*)

Các giá trị ở đầu bảng là những đặc trưng quan trọng nhất, trong khi các giá trị ở cuối bảng có ảnh hưởng ít nhất.

Số đầu tiên trong mỗi hàng thể hiện mức độ suy giảm hiệu suất của mô hình khi một đặc trưng bị xáo trộn ngẫu nhiên (trong trường hợp này, chúng ta sử dụng "độ chính xác" (*accuracy*) làm thước đo hiệu suất).

Như hầu hết các khía cạnh khác trong khoa học dữ liệu, có một chút ngẫu nhiên trong việc thay đổi hiệu suất khi xáo trộn một cột. Chúng ta đo lường mức độ ngẫu nhiên này bằng cách lặp lại quá trình xáo trộn nhiều lần. Số liệu sau ký hiệu **±** thể hiện mức độ biến thiên của hiệu suất giữa các lần xáo trộn.

Đôi khi, bạn có thể thấy giá trị âm trong tầm quan trọng hoán vị. Điều này xảy ra khi dự đoán trên dữ liệu bị xáo trộn (hoặc có nhiễu) tình cờ chính xác hơn so với dữ liệu thực tế. Điều này thường xảy ra khi đặc trưng đó thực sự không quan trọng (đáng lẽ có giá trị tầm quan trọng gần bằng 0), nhưng do may mắn mà mô hình lại đưa ra dự đoán tốt hơn trên dữ liệu bị xáo trộn. Hiện tượng này phổ biến hơn với các tập dữ liệu nhỏ, như tập dữ liệu trong ví dụ này, vì mức độ ảnh hưởng của sự may mắn lớn hơn.

Trong ví dụ của chúng ta, đặc trưng quan trọng nhất là **Số bàn thắng ghi được** (*Goals scored*). Điều này có vẻ hợp lý. Những người hâm mộ bóng đá có thể sẽ có trực giác về thứ tự của các đặc trưng khác liệu có đáng ngạc nhiên hay không.

## Đến Lượt Bạn

**[Bắt đầu ngay tại đây](https://www.kaggle.com/kernels/fork/1637562)** để áp dụng kiến thức mới về tầm quan trọng hoán vị của bạn.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/machine-learning-explainability/discussion) to chat with other learners.*